# Python Typing -- Selected Topics

* Based on:
  * [PEP 483 -- The Theory of Type Hints](https://www.python.org/dev/peps/pep-0483)
  
  * [PEP 484 -- Type Hints](https://www.python.org/dev/peps/pep-0484)
  
  * Python Typing [docs](https://docs.python.org/3/library/typing.html) 
  
  * mypy [docs](https://mypy.readthedocs.io/en/latest/)
  
  * mypy [github issues](https://github.com/python/mypy/issues)
  
  * some experience


* Using:
  * **Python 3.6**
  * **mypy==0.600** (official type checker, there is also Facebook's [pyre-check](https://github.com/facebook/pyre-check))

## Type vs Class

* Type is a type checker concept.

* Class is a runtime concept.

**Types can be defined:**

* By explicitly listing all values. E.g., `True` and `False` form the type `bool`.


* By a simple class definition, for example if one defines a class:

    ```python
    class UserID(int):
        pass
    ```
then all instances of this class also form a type.


* By specifying functions which can be used with variables of a type. E.g. all objects that have a `__len__` method form the type `Sized`. Both `[1, 2, 3]` and 'abc' belong to this type, since one can call len on them:

```python
len([1, 2, 3])  # OK
len('abc')      # also OK
len(42)         # not a member of Sized
```


* There are also more complex types. E.g., one can define the type `FancyList` as all lists containing only instances of `int`, `str` or their subclasses. The value `[1, 'abc', UserID(42)]` has this type.

[source](https://www.python.org/dev/peps/pep-0483/#background)

## Subtyping

Let `<:` mean "is a subtype of". (So `A <: B` reads "`A` is a subtype of `B`".)

`B <: A` iff:

1. every value from `B` is also in the set of values of `A`; and

2. every function from `A` is also in the set of functions of `B`.

So the set of values becomes smaller in the process of subtyping, while the set of functions becomes larger. 

`Dog <: Animal`:

1. There are less `Dog`s than `Animal`s (every `Dog` is an `Animal`, but not other way around).

2. `Animal`s can do less than `Dog`s (`Dog` can do whatever `Animal` can, but not other way around). `Animal`s can't bark like `Dog`s, but both can eat and breathe.

In [ ]:
class Animal:
    def eat(self): pass
    def breathe(self): pass

class Dog(Animal):
    def bark(self): pass

### Annotating variables

`<variable>: <type> = <value>`

In [ ]:
lassie: Dog

In [ ]:
some_animal: Animal

No values assigned, yes. There is no `lassie` and `some_animal` in runtime (using either would raise `NameError`). It's for type checker -- now it knows types of both.

### Assigning rules
If `Dog <: Animal` (as in our case):
- it is safe to assign: `some_animal = lassie`

- it is unsafe to assign `lassie = some_animal`, **mypy** complains in this case:

```
error: Incompatible types in assignment (expression has type "Animal", variable has type "Dog")
```

### Annotating functions

```python
def function(attr1: Type1, attr2: Type2) -> ReturnType: ...
```

In [ ]:
def add_ints(x: int, y: int) -> int:
    return x + y

add_ints(1, 2)    # no mypy error
add_ints(1, 2.0)  # error: Argument 2 to "add_ints" has incompatible type "float"; expected "int"

In [ ]:
def broken_add(x: int, y: int) -> str:
    return x + y

# error: Incompatible return value type (got "int", expected "str")

## Building blocks

### `Any`

`Any` is consistent with every type and every type is consistent with `Any`.

> [W]hen a value has type Any, the type checker will allow all operations on it, and a value of type Any can be assigned to a variable (or used as a return value) of a more constrained type. 

[https://www.python.org/dev/peps/pep-0484/#the-any-type]

In [ ]:
from typing import Any

In [ ]:
# checking types

lassie: Dog
anything: Any
    
lassie = anything  # no mypy error

In [ ]:
lassie: Dog
anything: Any
    
anything = lassie  # no mypy error

In [ ]:
# checking attributes

lassie: Dog
lassie.bark()  # no mypy error
lassie.fly()   # error: "Dog" has no attribute "fly"

anything: Any
anything.enter_hiperspace()  # no mypy error

> `Any` can be considered a type that has all values and all methods. Combined with the definition of subtyping above, this places `Any` partially at the top (it has all values) and bottom (it has all methods) of the type hierarchy.

[https://www.python.org/dev/peps/pep-0483/#summary-of-gradual-typing]

```
                      Any                     <- everything is Any                             
                      / \                        (like object               
                     /*  \*                       -- everything is object)                 
                    /     \                                         
            SomeType1     SomeType2                                   
           /       |       |       \                                   
          /        |       |        \                                  
         /         |       |         \                                 
 Subtype1_1  Subtype1_2  Subtype2_1  Subtype2_2                           
     |          |           |           |                              
     |*         |*          |*          |*                             
     |          |           |           |                              
    Any        Any         Any         Any       <- Any has all attributes
                                                     (unlike object
                                                      -- object has no attrs)
                                                      
* consistency relationship, not subtyping, see below
```

Mind that relation between `Any` and other types is not subtyping relationship but being-consistent-with relationship. For formal definition and more context see https://www.python.org/dev/peps/pep-0483/#summary-of-gradual-typing.

### `Union`

> `Union[t1, t2, ...]`. 
> Types that are subtype of at least one of t1 etc. are subtypes of this.

In [ ]:
from typing import Union

In [ ]:
class Animal:
    def eat(self): pass
    
class Dog(Animal): pass   
class Cat(Animal): pass
class Lizard(Animal): pass


# only Cats and Dogs are allowed to eat :(
def restricted_eat(animal: Union[Dog, Cat]) -> None:
    animal.eat()
    
a_dog: Dog
restricted_eat(a_dog)  # no mypy error

a_cat: Cat
restricted_eat(a_cat)  # no mypy error

a_lizard: Lizard
restricted_eat(a_lizard)  #  error: Argument 1 to "restricted_eat" has incompatible type "Lizard"; expected "Union[Dog, Cat]"

# KEEP IN MIND THAT NOT RUNTIME CHECKS ARE MADE HERE!

### `Optional` and constraining types

> `Optional[t1]`. Alias for `Union[t1, None]`, i.e. `Union[t1, type(None)]`.

In [ ]:
from typing import Optional

def get_user_id() -> Optional[int]:
    pass

def process_user_id(user_id: int):
    pass

user_id = get_user_id()
process_user_id(user_id)  # error: 
# Argument 1 to "process_pk" has incompatible type "Optional[int]"; expected "int"

If we are, for some reason, sure that `get_pk()` will return an `int` (from context or ne understand from our code more than type checker) we can inform type checker of it.

#### Type constraining with assertions

In [ ]:
user_id = get_user_id()

assert isinstance(user_id, int)
# or: user_id is not None
process_user_id(user_id)  # no mypy error

This introduces small runtime overhead. In bonus we get additional runtime checks.

#### Type constraining with type casting

### `List` and `Tuple`/`NamedTuple`

In [ ]:
from typing import List

my_list: List[int] = [1, 2, 3]  # no mypy error
my_other_list: List[int] = [1, 2, '3']  #  error: List item 2 has incompatible type "str"; expected "int"

In [ ]:
from typing import Tuple

bob: Tuple[str, str, int] = ('Bob', 'Smith', 25)  # no mypy error
    
frank: Tuple[str, str, int] = ('Frank', 'Brown', 43.4)  # error:
# Incompatible types in assignment (expression has type "Tuple[str, str, float]", variable has type "Tuple[str, str, int]")

ann: Tuple[str, str, int] = ('Ann', 'X', 1, 2)  # error: 
# Incompatible types in assignment (expression has type "Tuple[str, str, int, int]", variable has type "Tuple[str, str, int]")

# we can use ellipsis to define type repetition
immutable_list: Tuple[int, ...] = (5, 8, 4, -1)  # no mypy error

In [ ]:
from typing import NamedTuple

class Person(NamedTuple):
    first_name: str
    last_name: str
    age: int
        
Person('Kate', 'Smith', '32')  # error: Argument 3 to "Person" has incompatible type "str"; expected "int"

### ABCs/protocols

In [ ]:
from typing import Collection, Sequence

names1: Collection = {'Frank', 'Bob', 'Kate'}  # no mypy error

names2: Sequence = {'Frank', 'Bob', 'Kate'}  # error:
# Incompatible types in assignment (expression has type "Set[str]", variable has type "Sequence[Any]")

In [ ]:
from typing import Container

class C:
    def __contains__(self, obj) -> bool:
        pass    
c1: Container = C()  # no mypy error

c2: Container = object()  # error:
# Incompatible types in assignment (expression has type "object", variable has type "Container[Any]")

### Aliases

In [ ]:
from typing import Iterable, Tuple

FirstName = str
LastName = str
Age = int
Employee = Tuple[FirstName, LastName, Age]
Employees = Iterable[Employee]

employees1: Employees = [
    ('Bob', 'Smith', 25),
    ('Kate', 'Brown', 32),
]  # no mypy errors

employees2: Employees = [
    ('Frank', 'Smith', '25'),
]  # error: List item 0 has incompatible type "Tuple[str, str, str]"; expected "Tuple[str, str, int]"

### `NewType` helper function

Let's say we have `user_id` which is an int, but we don't want to mix it up with other ints. 

In [ ]:
def update_height(user_id: int, new_height: int):
    pass

user_id = 1138
new_height = 44
update_height(new_height, user_id)  # no mypy error, yet the code is wrong

#### Approach 1

In [ ]:
class UserId(int):
    pass

def update_height(user_id: UserId, new_height: int):
    pass

user_id = UserId(1138)
new_height = 44
update_height(new_height, user_id) # error: 
# Argument 1 to "update_height" has incompatible type "int"; expected "UserId"

#### Approach 2

Unfortunately passing value through a class introduces runtime overhead. `NewType` deals with it:

In [ ]:
from typing import NewType

UserId = NewType('UserId', int)

def update_height(user_id: UserId, new_height: int):
    pass

user_id = UserId(1138)
new_height = 44
update_height(new_height, user_id)  #  error:
# Argument 1 to "update_height" has incompatible type "int"; expected "UserId"

Minimal overhead in runtime, since `NewType` simply returns identity function, with almost zero overhead:

```python
def NewType(name, tp):
    def new_type(x):
        return x

    new_type.__name__ = name
    new_type.__supertype__ = tp
    return new_type
```

[https://github.com/python/typing/blob/master/src/typing.py#L2210-L2234